In [1]:
import pyspark
import findspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (SparkSession.builder
         .master("local[4]")
         .appName("test")
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/29 15:54:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!wget http://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2024-02-29 15:54:07--  http://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.133.88, 52.216.48.112, 52.216.170.221, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.133.88|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv.1’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0.003s  

2024-02-29 15:54:07 (4.28 MB/s) - ‘taxi+_zone_lookup.csv.1’ saved [12322/12322]



In [5]:
df = spark.read.csv("taxi+_zone_lookup.csv", header=True, inferSchema=True)

In [6]:
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [8]:
df.write.parquet('zones', mode='overwrite')

# Read FHVHV data

In [12]:
from pyspark import SparkFiles

file_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz'
spark.sparkContext.addFile(file_url)

df = spark.read.csv(SparkFiles.get('fhvhv_tripdata_2021-01.csv.gz'), inferSchema=True, header=True)

24/02/29 16:03:36 WARN SparkContext: The path https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz has been added already. Overwriting of added paths is not supported in the current version.


In [13]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   NULL|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [14]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [15]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', IntegerType(), True)])

## Infer schema by reading first 100 rows into pandas and then converting to spark making small adjustments

In [36]:
import pandas as pd

# Read the first 100 rows using Pandas
first_100_rows = pd.read_csv(file_url, compression='gzip', nrows=100, parse_dates=['pickup_datetime', 'dropoff_datetime'])

In [37]:
first_100_rows.dtypes

hvfhs_license_num               object
dispatching_base_num            object
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
SR_Flag                        float64
dtype: object

In [38]:
last_100_rows = pd.read_csv(file_url, compression='gzip', nrows=100, parse_dates=['pickup_datetime', 'dropoff_datetime'])


In [39]:
from pyspark.sql.types import IntegerType

In [40]:
first_100_rows_sp = spark.createDataFrame(first_100_rows)
first_100_rows_sp = (first_100_rows_sp
                     .withColumn("SR_Flag", first_100_rows_sp["SR_Flag"].cast(IntegerType()))
                     .withColumn("DOLocationID", first_100_rows_sp["DOLocationID"].cast(IntegerType()))
                     .withColumn("PULocationID", first_100_rows_sp["PULocationID"].cast(IntegerType())))
first_100_rows_sp.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', IntegerType(), True)])

## Read FHVHV data again providing the schema

In [41]:
df = spark.read.csv(SparkFiles.get('fhvhv_tripdata_2021-01.csv.gz'), header=True, schema=first_100_rows_sp.schema)

In [42]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

## Write the obtained csv to partitioned parquet

In [43]:
df.repartition(24).write.parquet('fhvhv/2021/01/')

We can see that resulting parquet files are considerably smaller than the original csv

In [47]:
!ls -lh fhvhv/2021/01/

total 461272
-rw-r--r--@ 1 alexlitvinov  staff     0B Feb 29 20:23 _SUCCESS
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Feb 29 20:23 part-00000-c61329de-c6e9-4b09-83e0-872bb659fe6f-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Feb 29 20:23 part-00001-c61329de-c6e9-4b09-83e0-872bb659fe6f-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Feb 29 20:23 part-00002-c61329de-c6e9-4b09-83e0-872bb659fe6f-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Feb 29 20:23 part-00003-c61329de-c6e9-4b09-83e0-872bb659fe6f-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Feb 29 20:23 part-00004-c61329de-c6e9-4b09-83e0-872bb659fe6f-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Feb 29 20:23 part-00005-c61329de-c6e9-4b09-83e0-872bb659fe6f-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Feb 29 20:23 part-00006-c61329de-c6e9-4b09-83e0-872bb659fe6f-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   9.4M Feb 29 20:23 part-00007-c6132

## Read repartitioned df back into spark

In [44]:
df = spark.read.parquet('fhvhv/2021/01/')

In [45]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: int]

In [46]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)



In [48]:
(df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')
 .filter(df.hvfhs_license_num == 'HV0003')
 .show())

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-12 09:46:38|2021-01-12 09:59:31|           4|         137|
|2021-01-22 18:07:54|2021-01-22 18:13:25|         242|          78|
|2021-01-19 08:55:37|2021-01-19 09:09:36|         244|         128|
|2021-01-12 04:16:21|2021-01-12 04:37:52|         230|         225|
|2021-01-20 17:00:35|2021-01-20 17:12:54|         228|          26|
|2021-01-29 17:27:39|2021-01-29 18:03:28|         123|         130|
|2021-01-16 14:47:42|2021-01-16 15:06:44|          47|         119|
|2021-01-03 05:33:48|2021-01-03 06:04:01|          14|          75|
|2021-01-10 16:31:19|2021-01-10 16:36:52|         150|         150|
|2021-01-30 17:44:41|2021-01-30 17:52:11|         205|         205|
|2021-01-15 15:15:26|2021-01-15 15:40:27|         129|         145|
|2021-01-14 09:18:22|2021-01-14 09:45:49|       

## Pyspark in-built sql functions

In [49]:
from pyspark.sql import functions as F

In [52]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [53]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [54]:
(df
 .withColumn('pickup_date', F.to_date(df.pickup_datetime))
 .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) 
 .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num))
 .show()
)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dropoff_date|base_id|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+-------+
|           HV0005|              B02510|2021-01-25 18:08:34|2021-01-25 18:20:34|         247|         169|   NULL| 2021-01-25|  2021-01-25|  e/9ce|
|           HV0005|              B02510|2021-01-01 06:52:41|2021-01-01 06:59:34|          14|         227|   NULL| 2021-01-01|  2021-01-01|  e/9ce|
|           HV0005|              B02510|2021-01-29 10:47:17|2021-01-29 11:06:12|         134|         138|   NULL| 2021-01-29|  2021-01-29|  e/9ce|
|           HV0003|              B02866|2021-01-12 09:46:38|2021-01-12 09:59:31|           4|         137|   NUL

### Read some yellow csv

In [56]:
yellow_file_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-06.csv.gz'
spark.sparkContext.addFile(yellow_file_url)

In [57]:
df_yellow = spark.read.csv(SparkFiles.get('yellow_tripdata_2019-06.csv.gz'), header=True, inferSchema=True)

In [61]:
df_yellow = df_yellow.withColumn('service_type', F.lit('yellow'))

In [64]:
df_yellow.createOrReplaceTempView('yellow_trips_june_2019')

In [69]:
spark.sql("""
select 
    service_type,
    count(1)
from yellow_trips_june_2019
group by service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|      yellow| 6941024|
+------------+--------+



In [70]:
df_yellow

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp, tpep_dropoff_datetime: timestamp, passenger_count: int, trip_distance: double, RatecodeID: int, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: int, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, service_type: string]

## Group by

In [ ]:
df_res = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow_trips_june_2019
GROUP BY
    1, 2
""").show()